# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/data_prep.ipynb)をお使い下さい。

In [1]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

Using TensorFlow backend.


In [2]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install smdebug

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!docker build -t sm-tf-nightly-gpu container/.

Sending build context to Docker daemon  13.31kB
Step 1/4 : FROM tensorflow/tensorflow:nightly-gpu
 ---> ad7cd4026486
Step 2/4 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 28545073ce2f
Step 3/4 : COPY tf_codes /opt/ml/code/
 ---> Using cache
 ---> f756d6396bd2
Step 4/4 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> b94cf342c274
Successfully built b94cf342c274
Successfully tagged sm-tf-nightly-gpu:latest


## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [4]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t word2vec .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sm-tf-nightly-gpu' already exists in the registry with id '815969174475'
unable to prepare context: unable to evaluate symlinks in Dockerfile path: lstat /home/ec2-user/SageMaker/sagemaker-tensorflow-byoc/Dockerfile: no such file or directory
The push refers to repository [815969174475.dkr.ecr.us-east-1.amazonaws.com/sm-tf-nightly-gpu]

4cc0702b: Preparing 
dd9cc193: Preparing 
ff51c6a3: Preparing 
753ea774: Preparing 
b74783e3: Preparing 
8916be48: Preparing 
0a0d84e3: Preparing 
97f02d3e: Preparing 
45052e8d: Preparing 
eb49a301: Preparing 
f76fc560: P

## 実験の作成

In [28]:
import boto3
import time
from smexperiments.experiment import Experiment

boto3_sess = boto3.Session()
sm = boto3_sess.client('sagemaker')

tf_experiment = Experiment.create(
    experiment_name=f"tensorflow-byoc-{int(time.time())}", 
    description="tensorflow experiments", 
    sagemaker_boto_client=sm)

print(tf_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f1682a76f28>,experiment_name='tensorflow-byoc-1582523849',description='tensorflow experiments',experiment_arn='arn:aws:sagemaker:us-east-1:815969174475:experiment/tensorflow-byoc-1582523849',response_metadata={'RequestId': 'e69a22fc-4cfa-4a4c-9423-126676e900bf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e69a22fc-4cfa-4a4c-9423-126676e900bf', 'content-type': 'application/x-amz-json-1.1', 'content-length': '98', 'date': 'Mon, 24 Feb 2020 05:57:29 GMT'}, 'RetryAttempts': 0})


### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。
ここでは Optimizer を 'Adam' と 'SGD' とで変更した際の実験管理を行っています。

In [29]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent


role = get_execution_role()
bucket = '20200222-tensorflow-byoc'
output_path = 's3://{}/output'.format(bucket)
data_path = {'train': 's3://{}/data'.format(bucket)}

metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'accuracy: ([0-9\\.]+)'}]

trial_name_map = {}

optimizers = ['adam', 'sgd']

for i, optimizer in enumerate(optimizers):
    # Trial の作成
    trial_name = f'tf-training-job-{optimizer}-{int(time.time())}'
    print(trial_name)
    
    
    tf_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=tf_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    trial_name_map[optimizer] = trial_name
    
    # Estimator の作成とfit。
    hyperparameters = {'batch_size': 64,'epochs': 1, 'optimizer': optimizer}
    estimator = Estimator(image_name=image_uri,
                          role=role,
                          hyperparameters=hyperparameters,
                          train_instance_count=1,
                          train_instance_type='ml.p2.xlarge',
                          output_path=output_path,
                          metric_definitions=metric_definitions,
                          enable_sagemaker_metrics=True,)
    
    tf_training_job_name = "tf-job-{}".format(int(time.time()))
    
    estimator.fit(data_path,
                 job_name=tf_training_job_name,
                  
                 experiment_config={
                     'TrialName': tf_trial.trial_name,
                     'TrialComponentDisplayName': 'Training',
                 },
                  
                 wait=False)
    
    time.sleep(2)

INFO:sagemaker:Creating training-job with name: tf-job-1582523851


tf-training-job-adam-1582523850


INFO:sagemaker:Creating training-job with name: tf-job-1582523853


tf-training-job-sgd-1582523853


In [35]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [36]:
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

sess = boto3.Session()

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=tf_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.accuracy.max",
    sort_order="Descending",
    metric_names=['accuracy'],
    parameter_names=['optimizer']
)

In [37]:
analytic_table = trial_component_analytics.dataframe()

In [38]:
analytic_table

,TrialComponentName,DisplayName,SourceArn,optimizer,accuracy - Min,accuracy - Max,accuracy - Avg,accuracy - StdDev,accuracy - Last,accuracy - Count
0,tf-job-1582523851-aws-training-job,Training,arn:aws:sagemaker:us-east-1:815969174475:train...,adam,0.1094,0.9375,0.52345,0.585555,0.9375,2
1,tf-job-1582523853-aws-training-job,Training,arn:aws:sagemaker:us-east-1:815969174475:train...,sgd,0.1562,0.3750,0.26560,0.154715,0.3750,2
